In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras
import os
import PIL

In [15]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

# Load dataset (Example: ImageNet or Custom)
dataset = datasets.ImageFolder(root="/content/drive/MyDrive/data_HAM/train",transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True,num_workers=8)

# Fetch a batch
images, labels = next(iter(dataloader))
print(images.shape, labels.shape)  # Should print: (batch_size, 3, 224, 224)


torch.Size([128, 3, 224, 224]) torch.Size([128])


In [26]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

])

# Load dataset (Example: ImageNet or Custom)
dataset = datasets.ImageFolder(root="/content/drive/MyDrive/data_HAM/val",transform=transform)
val_dataloader = DataLoader(dataset, batch_size=128, shuffle=True,num_workers=8)

# Fetch a batch
images, labels = next(iter(dataloader))
print(images.shape, labels.shape)  # Should print: (batch_size, 3, 224, 224)


torch.Size([128, 3, 224, 224]) torch.Size([128])


In [4]:
images[0].shape

torch.Size([3, 224, 224])

In [22]:
import timm
import torch
import torch.optim as optim  # Import optimizer

# Load a pre-trained ViT model
model = timm.create_model("vit_base_patch32_224", pretrained=True)

# Modify classifier if needed (replace your_num_classes)
model.head = torch.nn.Linear(in_features=model.head.in_features, out_features=7)

In [23]:
import torch.optim as optim

# Freeze early layers (optional)
for param in model.parameters():
    param.requires_grad = False
for param in model.head.parameters():  # Unfreeze the classifier head
    param.requires_grad = True

# Define optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate
criterion = torch.nn.CrossEntropyLoss()

# Training loop
# ... (similar to your existing training loop)

In [24]:
!pip install tqdm
from tqdm import tqdm

In [27]:
num_epochs = 5
for epoch in range(num_epochs):
    # Training loop
    model.train()  # Set model to training mode
    train_loss = 0
    correct_train = 0
    total_train = 0

    for batch_idx, (data, target) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Training")):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        data, target = data.to(device), target.to(device)

        # Move the model to the device before the forward pass
        model.to(device)  # This line is added to move the model to the GPU

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = output.max(1)
        total_train += target.size(0)
        correct_train += predicted.eq(target).sum().item()

    train_accuracy = 100. * correct_train / total_train

    # Validation loop (assuming you have a separate validation dataloader)
    model.eval()  # Set model to evaluation mode
    val_loss = 0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(tqdm(val_dataloader, desc=f"Epoch {epoch + 1}/{num_epochs} - Validation")):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            val_loss += loss.item()
            _, predicted = output.max(1)
            total_val += target.size(0)
            correct_val += predicted.eq(target).sum().item()

    val_accuracy = 100. * correct_val / total_val

    print(f"Epoch {epoch + 1}/{num_epochs}:")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

Epoch 1/5 - Validation: 100%|██████████| 16/16 [00:18<00:00,  1.14s/it]


Epoch 1/5:
  Train Loss: 39.0128, Train Acc: 77.22%
  Val Loss: 10.8373, Val Acc: 74.86%


Epoch 2/5 - Validation: 100%|██████████| 16/16 [00:18<00:00,  1.13s/it]


Epoch 2/5:
  Train Loss: 37.0367, Train Acc: 78.51%
  Val Loss: 10.5281, Val Acc: 75.31%


Epoch 3/5 - Validation: 100%|██████████| 16/16 [00:18<00:00,  1.13s/it]


Epoch 3/5:
  Train Loss: 35.3327, Train Acc: 79.75%
  Val Loss: 10.7085, Val Acc: 74.86%


Epoch 4/5 - Validation: 100%|██████████| 16/16 [00:18<00:00,  1.18s/it]


Epoch 4/5:
  Train Loss: 34.6761, Train Acc: 80.13%
  Val Loss: 10.2449, Val Acc: 76.10%


Epoch 5/5 - Validation: 100%|██████████| 16/16 [00:17<00:00,  1.11s/it]

Epoch 5/5:
  Train Loss: 32.9480, Train Acc: 81.16%
  Val Loss: 10.0447, Val Acc: 76.65%


In [28]:
class_names=['class1','class2','class3','class4','class5','class6','class7',]

In [29]:
from PIL import Image

In [34]:
image_path = "/content/drive/MyDrive/data_HAM/val/class6/ISIC_0025898.jpg"

image = Image.open(image_path).convert("RGB") # Ensure the image is in RGB format

# Preprocess the image using the torchvision transform you defined earlier
image_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# Move inputs to the device
image_tensor = image_tensor.to(device)

# Make a prediction
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    outputs = model(image_tensor)

    predicted_class_index = outputs.argmax(-1).item()

# Get the class names (assuming you have a list of class names)
class_names = dataset.classes  # Or replace with your class names

# Print the predicted class
predicted_class_name = class_names[predicted_class_index]
print(f"Predicted class: {predicted_class_name}")

Predicted class: class6


it got around 80 % accuracy which is good but still can be improved by unfreezing some layers and train it accorgingly